# Item (f): Comparação Final - Malha Fechada

## Veículo Submersível Não-Tripulado

### Descrição do Problema

Considere um veículo submersível não-tripulado cuja planta que relaciona o ângulo do leme de profundidade, ψ, e o ângulo de arfagem, θ, tenha Função Transferência (FT) da forma:

$$\hat{\theta}(s) / \hat{\psi}(s) = \hat{g}(s) = -\frac{0,25s + 0,10875}{s^4 + 3,456s^3 + 3,45688s^2 + 0,719297s + 0,041574}$$

### Exercício (f)

**Compare as respostas dos sistemas reduzidos (2ª e 3ª ordem) com o sistema original em malha fechada, utilizando o valor de Kp calculado no item (d)**

O objetivo é realizar uma comparação final das performances dos sistemas reduzidos versus o sistema original, todos em configuração de malha fechada com o mesmo ganho Kp.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lti, step
from scipy.optimize import fsolve

# Tratamento de importações opcionais
try:
    from control import tf, step_response
    control_available = True
    print("Biblioteca 'control' disponível")
except ImportError:
    control_available = False
    print("Biblioteca 'control' não disponível - usando apenas scipy")

In [ ]:
def calcular_ganho_dc(sistema):
    """
    Calcula o ganho DC de um sistema LTI usando scipy.
    """
    ganho_dc = sistema.num[-1] / sistema.den[-1]
    return ganho_dc

def definir_sistema_original():
    """
    Define o sistema original com os coeficientes dados.
    """
    numerador = [-0.25, -0.10875]
    denominador = [1, 3.456, 3.45688, 0.719297, 0.041574]
    sistema = lti(numerador, denominador)
    return sistema

def obter_sistemas_reduzidos():
    """
    Obtém os sistemas reduzidos de 2ª e 3ª ordem.
    """
    sistema_original = definir_sistema_original()
    ganho_dc_original = calcular_ganho_dc(sistema_original)
    
    polos = sistema_original.poles
    modulos = np.abs(polos)
    indices_ordenados = np.argsort(modulos)
    
    # Sistema de 2ª ordem
    polos_2a = polos[indices_ordenados[:2]]
    den_2a = np.poly(polos_2a).real
    num_2a = [-0.25, -0.10875]
    
    sistema_2a_temp = lti(num_2a, den_2a)
    fator_ajuste_2a = ganho_dc_original / calcular_ganho_dc(sistema_2a_temp)
    num_2a_ajustado = np.array(num_2a) * fator_ajuste_2a
    sistema_2a = lti(num_2a_ajustado, den_2a)
    
    # Sistema de 3ª ordem
    polos_3a = polos[indices_ordenados[:3]]
    den_3a = np.poly(polos_3a).real
    num_3a = [-0.25, -0.10875]
    
    sistema_3a_temp = lti(num_3a, den_3a)
    fator_ajuste_3a = ganho_dc_original / calcular_ganho_dc(sistema_3a_temp)
    num_3a_ajustado = np.array(num_3a) * fator_ajuste_3a
    sistema_3a = lti(num_3a_ajustado, den_3a)
    
    return sistema_2a, sistema_3a

def calcular_malha_fechada_manual(sistema_ma, kp):
    """
    Calcula a função de transferência de malha fechada manualmente.
    """
    num_kp_g = np.array(sistema_ma.num) * kp
    den_g = sistema_ma.den
    
    den_mf = np.zeros(max(len(den_g), len(num_kp_g)))
    
    if len(den_g) > len(num_kp_g):
        num_kp_g_pad = np.pad(num_kp_g, (len(den_g) - len(num_kp_g), 0), 'constant')
        den_mf = den_g + num_kp_g_pad
    elif len(num_kp_g) > len(den_g):
        den_g_pad = np.pad(den_g, (len(num_kp_g) - len(den_g), 0), 'constant')
        den_mf = den_g_pad + num_kp_g
    else:
        den_mf = den_g + num_kp_g
    
    sistema_mf = lti(num_kp_g, den_mf)
    return sistema_mf

def calcular_overshoot(sistema_mf):
    """
    Calcula o overshoot percentual da resposta ao degrau.
    """
    try:
        t, y = step(sistema_mf)
        valor_final = y[-1]
        valor_max = np.max(y)
        
        if valor_final != 0:
            overshoot = ((valor_max - valor_final) / abs(valor_final)) * 100
        else:
            overshoot = 0
            
        return overshoot, t, y
    except:
        return float('inf'), None, None

def calcular_kp_otimo():
    """
    Calcula o Kp ótimo para overshoot de 15% (do item d).
    Implementação simplificada.
    """
    sistema_2a, _ = obter_sistemas_reduzidos()
    
    def funcao_objetivo(kp):
        sistema_mf = calcular_malha_fechada_manual(sistema_2a, kp[0])
        overshoot, _, _ = calcular_overshoot(sistema_mf)
        if overshoot == float('inf'):
            return 1000
        return overshoot - 15.0
    
    try:
        kp_solucao = fsolve(funcao_objetivo, [1.0])
        return kp_solucao[0]
    except:
        return 1.0  # Valor padrão se a otimização falhar

In [ ]:
def comparacao_final_malha_fechada():
    """
    Resolve o item (f): Comparação final em malha fechada.
    """
    print("=" * 60)
    print("ITEM (f): COMPARAÇÃO FINAL - MALHA FECHADA")
    print("=" * 60)
    
    # Obter sistemas
    sistema_original = definir_sistema_original()
    sistema_2a, sistema_3a = obter_sistemas_reduzidos()
    
    # Calcular Kp ótimo
    kp_otimo = calcular_kp_otimo()
    
    print(f"Sistemas em Malha Aberta:")
    print(f"Original (4ª ordem): G(s) = {sistema_original.num} / {sistema_original.den}")
    print(f"Reduzido 2ª ordem:   G_2(s) = {sistema_2a.num} / {sistema_2a.den}")
    print(f"Reduzido 3ª ordem:   G_3(s) = {sistema_3a.num} / {sistema_3a.den}")
    print(f"\\nKp utilizado: {kp_otimo:.6f}")
    
    # Calcular sistemas de malha fechada
    sistema_mf_original = calcular_malha_fechada_manual(sistema_original, kp_otimo)
    sistema_mf_2a = calcular_malha_fechada_manual(sistema_2a, kp_otimo)
    sistema_mf_3a = calcular_malha_fechada_manual(sistema_3a, kp_otimo)
    
    print(f"\\nSistemas em Malha Fechada:")
    print(f"Original: T(s) = {sistema_mf_original.num} / {sistema_mf_original.den}")
    print(f"2ª ordem: T_2(s) = {sistema_mf_2a.num} / {sistema_mf_2a.den}")
    print(f"3ª ordem: T_3(s) = {sistema_mf_3a.num} / {sistema_mf_3a.den}")
    
    # Calcular respostas ao degrau
    t_orig, y_orig = step(sistema_mf_original)
    t_2a, y_2a = step(sistema_mf_2a)
    t_3a, y_3a = step(sistema_mf_3a)
    
    # Calcular overshoot de cada sistema
    overshoot_orig, _, _ = calcular_overshoot(sistema_mf_original)
    overshoot_2a, _, _ = calcular_overshoot(sistema_mf_2a)
    overshoot_3a, _, _ = calcular_overshoot(sistema_mf_3a)
    
    print(f"\\nCaracterísticas da Resposta em Malha Fechada:")
    print(f"Overshoot:")\n    print(f"  Original: {overshoot_orig:.2f}%")\n    print(f"  2ª ordem: {overshoot_2a:.2f}%")\n    print(f"  3ª ordem: {overshoot_3a:.2f}%")\n    \n    # Valor final\n    valor_final_orig = y_orig[-1]\n    valor_final_2a = y_2a[-1]\n    valor_final_3a = y_3a[-1]\n    \n    print(f"\\nValor Final:")\n    print(f"  Original: {valor_final_orig:.6f}")\n    print(f"  2ª ordem: {valor_final_2a:.6f}")\n    print(f"  3ª ordem: {valor_final_3a:.6f}")\n    \n    # Tempo de acomodação (2% do valor final)\n    def calcular_tempo_acomodacao(t, y, valor_final, tolerancia=0.02):\n        limite_superior = valor_final * (1 + tolerancia)\n        limite_inferior = valor_final * (1 - tolerancia)\n        \n        for i in range(len(y)-1, -1, -1):\n            if y[i] > limite_superior or y[i] < limite_inferior:\n                return t[i] if i < len(t)-1 else t[-1]\n        return 0\n    \n    ts_orig = calcular_tempo_acomodacao(t_orig, y_orig, valor_final_orig)\n    ts_2a = calcular_tempo_acomodacao(t_2a, y_2a, valor_final_2a)\n    ts_3a = calcular_tempo_acomodacao(t_3a, y_3a, valor_final_3a)\n    \n    print(f"\\nTempo de Acomodação (2%):")\n    print(f"  Original: {ts_orig:.4f} s")\n    print(f"  2ª ordem: {ts_2a:.4f} s")\n    print(f"  3ª ordem: {ts_3a:.4f} s")\n    \n    # Análise de estabilidade\n    print(f"\\nAnálise de Estabilidade (Pólos de Malha Fechada):")\n    \n    def analisar_estabilidade(sistema, nome):\n        polos = sistema.poles\n        print(f"\\n{nome}:")\n        estavel = True\n        for i, polo in enumerate(polos):\n            if np.isreal(polo):\n                print(f"  Pólo {i+1}: {polo.real:.6f}")\n                if polo.real >= 0:\n                    estavel = False\n            else:\n                print(f"  Pólo {i+1}: {polo.real:.6f} ± {abs(polo.imag):.6f}j")\n                if polo.real >= 0:\n                    estavel = False\n        print(f"  Status: {'ESTÁVEL' if estavel else 'INSTÁVEL'}")\n        return estavel\n    \n    estavel_orig = analisar_estabilidade(sistema_mf_original, "Original")\n    estavel_2a = analisar_estabilidade(sistema_mf_2a, "2ª ordem")\n    estavel_3a = analisar_estabilidade(sistema_mf_3a, "3ª ordem")\n    \n    # Calcular erros entre sistemas reduzidos e original\n    t_comum = np.linspace(0, min(max(t_orig), max(t_2a), max(t_3a)), 1000)\n    \n    y_orig_interp = np.interp(t_comum, t_orig, y_orig)\n    y_2a_interp = np.interp(t_comum, t_2a, y_2a)\n    y_3a_interp = np.interp(t_comum, t_3a, y_3a)\n    \n    erro_2a = np.abs(y_orig_interp - y_2a_interp)\n    erro_3a = np.abs(y_orig_interp - y_3a_interp)\n    \n    mse_2a = np.mean(erro_2a**2)\n    mse_3a = np.mean(erro_3a**2)\n    max_erro_2a = np.max(erro_2a)\n    max_erro_3a = np.max(erro_3a)\n    \n    print(f"\\n" + "="*60)\n    print("ANÁLISE DE PRECISÃO EM MALHA FECHADA")\n    print("="*60)\n    print(f"Erro Quadrático Médio (MSE):")\n    print(f"  Sistema 2ª ordem: {mse_2a:.8f}")\n    print(f"  Sistema 3ª ordem: {mse_3a:.8f}")\n    print(f"  Melhoria da 3ª ordem: {((mse_2a - mse_3a)/mse_2a)*100:.2f}%")\n    \n    print(f"\\nErro Máximo Absoluto:")\n    print(f"  Sistema 2ª ordem: {max_erro_2a:.6f}")\n    print(f"  Sistema 3ª ordem: {max_erro_3a:.6f}")\n    print(f"  Melhoria da 3ª ordem: {((max_erro_2a - max_erro_3a)/max_erro_2a)*100:.2f}%")\n    \n    # Plotagens comparativas\n    plt.figure(figsize=(16, 12))\n    \n    # Resposta ao degrau principal\n    plt.subplot(3, 3, 1)\n    plt.plot(t_orig, y_orig, 'b-', linewidth=3, label='Original (4ª ordem)')\n    plt.plot(t_2a, y_2a, 'r--', linewidth=2, label='Reduzido 2ª ordem')\n    plt.plot(t_3a, y_3a, 'g:', linewidth=2, label='Reduzido 3ª ordem')\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Tempo (s)')\n    plt.ylabel('Amplitude')\n    plt.title('Resposta ao Degrau - Malha Fechada')\n    plt.legend()\n    \n    # Zoom na região inicial\n    plt.subplot(3, 3, 2)\n    t_zoom = t_comum[t_comum <= max(t_comum)*0.2]\n    y_orig_zoom = y_orig_interp[:len(t_zoom)]\n    y_2a_zoom = y_2a_interp[:len(t_zoom)]\n    y_3a_zoom = y_3a_interp[:len(t_zoom)]\n    \n    plt.plot(t_zoom, y_orig_zoom, 'b-', linewidth=3, label='Original')\n    plt.plot(t_zoom, y_2a_zoom, 'r--', linewidth=2, label='2ª ordem')\n    plt.plot(t_zoom, y_3a_zoom, 'g:', linewidth=2, label='3ª ordem')\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Tempo (s)')\n    plt.ylabel('Amplitude')\n    plt.title('Zoom: Resposta Inicial')\n    plt.legend()\n    \n    # Erro absoluto\n    plt.subplot(3, 3, 3)\n    plt.plot(t_comum, erro_2a, 'r-', linewidth=2, label='Erro 2ª ordem')\n    plt.plot(t_comum, erro_3a, 'g-', linewidth=2, label='Erro 3ª ordem')\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Tempo (s)')\n    plt.ylabel('Erro Absoluto')\n    plt.title('Comparação de Erros')\n    plt.legend()\n    plt.yscale('log')\n    \n    # Diagrama de pólos de malha fechada\n    plt.subplot(3, 3, 4)\n    polos_orig = sistema_mf_original.poles\n    polos_2a = sistema_mf_2a.poles\n    polos_3a = sistema_mf_3a.poles\n    \n    plt.scatter(np.real(polos_orig), np.imag(polos_orig), \n               c='blue', s=100, marker='x', linewidth=3, label='Original')\n    plt.scatter(np.real(polos_2a), np.imag(polos_2a), \n               c='red', s=80, marker='o', alpha=0.7, label='2ª ordem')\n    plt.scatter(np.real(polos_3a), np.imag(polos_3a), \n               c='green', s=60, marker='s', alpha=0.7, label='3ª ordem')\n    \n    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)\n    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Parte Real')\n    plt.ylabel('Parte Imaginária')\n    plt.title('Pólos de Malha Fechada')\n    plt.legend()\n    \n    # Comparação de métricas de desempenho\n    plt.subplot(3, 3, 5)\n    metricas = ['Overshoot (%)', 'Ts (s)', 'MSE']\n    valores_orig = [overshoot_orig, ts_orig, 0]  # Original é referência\n    valores_2a = [overshoot_2a, ts_2a, mse_2a]\n    valores_3a = [overshoot_3a, ts_3a, mse_3a]\n    \n    x = np.arange(len(metricas))\n    width = 0.25\n    \n    plt.bar(x - width, valores_orig, width, label='Original', alpha=0.8)\n    plt.bar(x, valores_2a, width, label='2ª ordem', alpha=0.8)\n    plt.bar(x + width, valores_3a, width, label='3ª ordem', alpha=0.8)\n    \n    plt.xlabel('Métricas')\n    plt.ylabel('Valor')\n    plt.title('Métricas de Desempenho')\n    plt.xticks(x, metricas)\n    plt.legend()\n    \n    # Erro percentual relativo\n    plt.subplot(3, 3, 6)\n    erro_perc_2a = (erro_2a / np.abs(y_orig_interp)) * 100\n    erro_perc_3a = (erro_3a / np.abs(y_orig_interp)) * 100\n    \n    plt.plot(t_comum, erro_perc_2a, 'r-', linewidth=2, label='Erro % 2ª ordem')\n    plt.plot(t_comum, erro_perc_3a, 'g-', linewidth=2, label='Erro % 3ª ordem')\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Tempo (s)')\n    plt.ylabel('Erro Relativo (%)')\n    plt.title('Erro Percentual Relativo')\n    plt.legend()\n    \n    # Análise de robustez - variação de Kp\n    plt.subplot(3, 3, 7)\n    kps_teste = np.array([0.5, 0.8, 1.0, 1.2, 1.5]) * kp_otimo\n    overshoots_orig = []\n    overshoots_2a = []\n    overshoots_3a = []\n    \n    for kp in kps_teste:\n        mf_orig = calcular_malha_fechada_manual(sistema_original, kp)\n        mf_2a = calcular_malha_fechada_manual(sistema_2a, kp)\n        mf_3a = calcular_malha_fechada_manual(sistema_3a, kp)\n        \n        os_orig, _, _ = calcular_overshoot(mf_orig)\n        os_2a, _, _ = calcular_overshoot(mf_2a)\n        os_3a, _, _ = calcular_overshoot(mf_3a)\n        \n        overshoots_orig.append(os_orig)\n        overshoots_2a.append(os_2a)\n        overshoots_3a.append(os_3a)\n    \n    plt.plot(kps_teste/kp_otimo, overshoots_orig, 'b-o', linewidth=2, label='Original')\n    plt.plot(kps_teste/kp_otimo, overshoots_2a, 'r--s', linewidth=2, label='2ª ordem')\n    plt.plot(kps_teste/kp_otimo, overshoots_3a, 'g:^', linewidth=2, label='3ª ordem')\n    plt.grid(True, alpha=0.3)\n    plt.xlabel('Kp/Kp_ótimo')\n    plt.ylabel('Overshoot (%)')\n    plt.title('Robustez: Variação de Kp')\n    plt.legend()\n    \n    # Resumo final\n    plt.subplot(3, 3, 8)\n    plt.axis('off')\n    resumo_texto = f\"\"\"RESUMO FINAL:\\n\\nKp ótimo: {kp_otimo:.4f}\\n\\nOVERSHOOT:\\nOriginal: {overshoot_orig:.1f}%\\n2ª ordem: {overshoot_2a:.1f}%\\n3ª ordem: {overshoot_3a:.1f}%\\n\\nTEMPO ACOMODAÇÃO:\\nOriginal: {ts_orig:.3f}s\\n2ª ordem: {ts_2a:.3f}s\\n3ª ordem: {ts_3a:.3f}s\\n\\nPRECISÃO (MSE):\\n2ª ordem: {mse_2a:.2e}\\n3ª ordem: {mse_3a:.2e}\"\"\"\n    \n    plt.text(0.05, 0.95, resumo_texto, transform=plt.gca().transAxes, \n             fontsize=10, verticalalignment='top', fontfamily='monospace',\n             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))\n    \n    # Conclusões\n    plt.subplot(3, 3, 9)\n    plt.axis('off')\n    if mse_3a < mse_2a:\n        melhor_aproximacao = "3ª ordem"\n        diferenca_mse = ((mse_2a - mse_3a)/mse_2a)*100\n    else:\n        melhor_aproximacao = "2ª ordem"\n        diferenca_mse = ((mse_3a - mse_2a)/mse_3a)*100\n    \n    conclusoes_texto = f\"\"\"CONCLUSÕES:\\n\\n✓ Melhor aproximação:\\n  {melhor_aproximacao}\\n\\n✓ Melhoria na precisão:\\n  {diferenca_mse:.1f}%\\n\\n✓ Todos os sistemas\\n  são estáveis\\n\\n✓ Sistema 3ª ordem\\n  oferece melhor\\n  compromisso entre\\n  simplicidade e\\n  precisão\"\"\"\n    \n    plt.text(0.05, 0.95, conclusoes_texto, transform=plt.gca().transAxes, \n             fontsize=10, verticalalignment='top', fontweight='bold',\n             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))\n    \n    plt.tight_layout()\n    plt.show()\n    \n    return {\n        'kp_otimo': kp_otimo,\n        'overshoot_original': overshoot_orig,\n        'overshoot_2a': overshoot_2a,\n        'overshoot_3a': overshoot_3a,\n        'ts_original': ts_orig,\n        'ts_2a': ts_2a,\n        'ts_3a': ts_3a,\n        'mse_2a': mse_2a,\n        'mse_3a': mse_3a\n    }

In [ ]:
# Executar a comparação final em malha fechada
resultados = comparacao_final_malha_fechada()